In [1]:
import numpy as np
import os
import glob
from python_speech_features import mfcc, logfbank, get_filterbanks
from python_speech_features.sigproc import preemphasis, framesig, powspec
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Masking, TimeDistributed, Conv1D
from keras.optimizers import SGD
from keras import metrics
import scipy.io.wavfile as wav
import sys
import subprocess
import h5py

/home/timo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
TEST_NAMES = ['aa', 'ae', 'ah', 'aw', 'ay', 'b', 'ch', 'd', 'dh', 'dx', 'eh', 'er', 'ey',
              'f', 'g', 'hh', 'ih', 'iy', 'jh', 'k', 'l', 'm', 'n', 'ng', 'ow', 'oy', 'p',
              'r', 's', 'sh', 'sil', 't', 'th', 'uh', 'uw', 'v', 'w', 'y', 'z']

FOLDINGS = {
    "ux": "uw",
    "axr": "er",
    "axh": "ah",
    "ax-h": "ah",
    "ax": "ah",
    "em": "m",
    "nx": "n",
    "en": "n",
    "eng": "ng",
    "hv": "hh",
    "el": "l",
    "zh": "sh",
    "ao": "aa",
    "ix": "ih",
    "pcl": "sil",
    "tcl": "sil",
    "kcl": "sil",
    "qcl": "sil",
    "bcl": "sil",
    "dcl": "sil",
    "gcl": "sil",
    "h#": "sil",
    "#h": "sil",
    "pau": "sil",
    "epi": "sil"
}

In [3]:
def phn_file_to_labels(fname, winstep=0.01, rate=16000.0):
    # Load PHN file
    phn_text = np.loadtxt(fname,
                          dtype={'names': ('start', 'end', 'phone'),
                                 'formats': (np.int32, np.int32, 'S4')},
                          comments=None)

    # Length of annotations in num samples
    _, phn_len, _ = phn_text[-1]
    labels = []
    for i, (start, end, phn) in enumerate(phn_text):
        # Round the start & end times to the window start
        start = int(start / (rate * winstep))
        end = int(end / (rate * winstep))
        labels.extend([phn.decode("utf-8")] * (end - start))

    return np.asarray(labels)


def fold_label(phn):
    if phn in FOLDINGS:
        return FOLDINGS[phn]
    else:
        return phn


def label_to_int(phn):
    return TEST_NAMES.index(phn)


def to_one_hot(labels, num_classes=48):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels.ravel()] = 1
    return labels_one_hot

In [4]:
source_dir = '../../TIMIT/TIMIT/TRAIN'
target_dir = '../../TIMIT/0_frames/TRAIN'

drs = sorted(glob.glob(os.path.join(source_dir, 'DR*')))
cmd = "sox {0} -t wav {1}"

#target_file_X = h5py.File(os.path.join(target_dir, 'features_framed.h5'), "w")
#target_file_Y = h5py.File(os.path.join(target_dir, 'features_preemph.h5'), "w")

winlen = 0.025
winstep = 0.01

X = []
Y = []

for dialect_region_dir in drs:
    print(dialect_region_dir)
    dr = os.path.split(dialect_region_dir)[-1]
    for speaker_dir in sorted(glob.glob(os.path.join(dialect_region_dir, "*"))):
        speaker = os.path.split(speaker_dir)[-1]
        print(speaker_dir)
        for audio_file in sorted(glob.glob(os.path.join(speaker_dir, "*.WAV"))):
            fname = os.path.splitext(os.path.split(audio_file)[-1])[0]
            if audio_file[8:10] == "SA":
                continue
            print(audio_file)
            file_target_dir = os.path.join(target_dir, dr, speaker)
            os.makedirs(file_target_dir, exist_ok=True)

            # Convert NIST wav files using sox
            converted_target_file = os.path.join(file_target_dir, os.path.split(audio_file)[-1])
            if not os.path.isfile(converted_target_file):
                subprocess.call(cmd.format(audio_file, converted_target_file), shell=True)

            (rate, sig) = wav.read(converted_target_file)
            #print(sig.shape)
            
            # Convert to overlapping frames
            x = framesig(sig, winlen * rate, winstep * rate)
            
            # Pre-emphasis
            sig_preemph = preemphasis(sig)
            
            # Convert to overlapping frames
            y = framesig(sig_preemph, winlen * rate, winstep * rate)
            
            for frame in x:
                X.append(frame)
            for frame in y:
                Y.append(frame)
            
            #group = target_file_X.create_group(fname)
            #group.create_dataset('features', data=x)
            #group2 = target_file_Y.create_group(fname)
            #group2.create_dataset('features', data=y)

../../TIMIT/TIMIT/TRAIN/DR1
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SI1027.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SI1657.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SI648.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SX127.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SX217.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SX307.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SX37.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FCJF0/DR1FCJF0SX397.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SI1271.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SI1406.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SI2036.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SX146.WAV
../../TIMIT/TIMIT/TRAIN/DR1/FDAW0/DR1FDAW0SX236.WAV
../../TIMIT/TIMIT/TRAIN/

../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SI1053.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SI1683.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SI552.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SX153.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SX243.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SX333.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SX423.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MDPK0/DR1MDPK0SX63.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SI1374.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SI2004.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SI744.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SX114.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SX204.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SX24.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SX294.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MEDR0/DR1MEDR0SX384.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MGRL

../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SI1235.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SI1865.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SI605.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SX155.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SX245.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SX335.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SX425.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTPF0/DR1MTPF0SX65.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SI1548.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SI2178.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SI918.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SX108.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SX18.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SX198.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SX288.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MTRR0/DR1MTRR0SX378.WAV
../../TIMIT/TIMIT/TRAIN/DR1/MWAD

../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SI1243.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SI1873.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SI613.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SX163.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SX253.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SX343.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SX433.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMA0/DR2FLMA0SX73.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SI1372.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SI2002.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SI742.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SX112.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SX22.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SX292.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/DR2FLMC0SX336.WAV
../../TIMIT/TIMIT/TRAIN/DR2/FLMC0/

../../TIMIT/TIMIT/TRAIN/DR2/MDBP0/DR2MDBP0SX168.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDBP0/DR2MDBP0SX258.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDBP0/DR2MDBP0SX348.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDBP0/DR2MDBP0SX438.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDBP0/DR2MDBP0SX78.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SI1868.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SI608.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SI800.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SX158.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SX248.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SX338.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SX428.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDEM0/DR2MDEM0SX68.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDLB0
../../TIMIT/TIMIT/TRAIN/DR2/MDLB0/DR2MDLB0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDLB0/DR2MDLB0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR2/MDLB0/DR2MDLB0SI1306.WAV
../.

...

../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SI1660.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SI2290.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SI650.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SX130.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SX220.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SX310.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SX40.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCLK0/DR7MCLK0SX400.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SI1121.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SI1725.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SI1751.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SX131.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SX221.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SX24.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SX401.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCRE0/DR7MCRE0SX41.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MCTH

../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SI1575.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SI2205.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SI945.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SX135.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SX225.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SX315.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SX405.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHBS0/DR7MHBS0SX45.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SI1772.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SI512.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SI612.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SX152.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SX242.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SX332.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/DR7MHXL0SX422.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MHXL0/

../../TIMIT/TIMIT/TRAIN/DR7/MRLJ1/DR7MRLJ1SI2332.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRLJ1/DR7MRLJ1SX141.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRLJ1/DR7MRLJ1SX231.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRLJ1/DR7MRLJ1SX321.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRLJ1/DR7MRLJ1SX411.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRLJ1/DR7MRLJ1SX51.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SI1080.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SI1710.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SI2340.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SX180.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SX270.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SX360.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SX450.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMG0/DR7MRMG0SX90.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMH0
../../TIMIT/TIMIT/TRAIN/DR7/MRMH0/DR7MRMH0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR7/MRMH0/DR7MRMH0SA2.WAV
..

../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SI1612.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SI2242.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SI982.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SX172.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SX262.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SX352.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SX442.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FBCG1/DR8FBCG1SX82.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SI1248.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SI1878.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SI618.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SX168.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SX258.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SX348.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0/DR8FCEG0SX438.WAV
../../TIMIT/TIMIT/TRAIN/DR8/FCEG0

../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SI559.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SI888.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SX168.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SX258.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SX348.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SX438.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MMWS0/DR8MMWS0SX78.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SA1.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SA2.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SI1044.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SI1595.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SI965.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SX155.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SX245.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SX335.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SX425.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRDM0/DR8MRDM0SX65.WAV
../../TIMIT/TIMIT/TRAIN/DR8/MRLK0
../../TIMIT/TIMIT/TRAIN/DR8/MRLK0/DR8MRLK0SA1.WAV
..

In [8]:
np.asarray(X).shape

(2397, 400)

In [33]:
def safe_log(features):
    features = np.where(features == 0, np.finfo(float).eps, features)
    return np.log(features)

def normalize(sequence):
    return (sequence - np.mean(sequence, axis=0)) / np.std(sequence, axis=0)

def median_normalize(sequence):
    return (sequence - np.median(sequence, axis=0))

def normalize_scale(sequence):
    return (sequence - np.min(sequence, axis=0)) / (np.max(sequence, axis=0) - np.min(sequence, axis=0))

class TimitTargetGenerator(object):
    def __init__(self, datafile_source, datafile_target, keys=None, batch_size=20, shuffle=True, mask_value=0.):
        self._datafile = datafile_source
        self._datafile_target = datafile_target
        self._batch_size = batch_size
        if keys is None:
            self._keys = list(datafile_source.keys())
        else:
            self._keys = keys
        self._batch_index = 0
        self._shuffle = shuffle
        self._mask_value = mask_value

    def shuffle_keys(self):
        random.shuffle(self._keys)

    def next_sample(self):
        key = self._keys[self._batch_index]
        self._batch_index += 1
        if self._batch_index == len(self._keys):
            self._batch_index = 0
            if self._shuffle:
                self.shuffle_keys()
        return key
    
    def pad_features(self, sequence, target_length):
        return np.pad(sequence,
                      ((0, target_length - sequence.shape[0]), (0, 0)), 'constant',
                      constant_values=((0, self._mask_value), (0, 0)))

    def pad_labels(self, sequence, target_length):
        return np.pad(sequence,
                      ((0, target_length - sequence.shape[0]), (0, 0)), 'constant',
                      constant_values=((0, 0), (0, 0)))
    
    def generator(self):
        while True:
            batch_keys = [self.next_sample() for _ in range(self._batch_size)]
            batch_features = [self._datafile[key]['features'][()] for key in batch_keys]
            batch_targets = [self._datafile_target[key]['features'][()] for key in batch_keys]
            
            yield batch_features, batch_targets

In [34]:
train_data_source_file = '../../TIMIT/0_frames/TRAIN/features_framed.h5'
train_data_target_file = '../../TIMIT/0_frames/TRAIN/features_preemph.h5'

train_data_source = h5py.File(train_data_source_file, 'r')
train_data_target = h5py.File(train_data_target_file, 'r')
train_generator = TimitTargetGenerator(train_data_source, train_data_target)

In [32]:
list(train_data_source['DR1FCJF0SA1']['features'])

[array([ 1., -1.,  2.,  0.,  0.,  3.,  0.,  1., -3., -2.,  2.,  1.,  0.,
         0.,  3.,  5.,  5.,  5.,  5.,  5.,  4.,  3.,  5.,  5.,  5.,  5.,
         4.,  1.,  5.,  0.,  2.,  6.,  4.,  5.,  4.,  6.,  4.,  2.,  2.,
        -3., -1., -2.,  4.,  5.,  1., -1., -2., -4., -1.,  4.,  3.,  6.,
         4.,  6.,  3.,  4.,  0.,  5.,  5.,  3., -3., -1.,  0., -3.,  1.,
         0., -3., -2.,  4., -2.,  0.,  4.,  2.,  3., -2., -3., -2.,  1.,
         4.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  1.,  1.,
        -2., -3., -2., -2., -2.,  1.,  3.,  5.,  5.,  5.,  5.,  5.,  5.,
         3.,  4.,  1.,  1.,  0.,  2.,  5.,  5.,  5.,  5.,  4.,  1.,  5.,
         4.,  3.,  6.,  1.,  5.,  5.,  5.,  2.,  5.,  5.,  5.,  6.,  5.,
         5.,  6.,  4.,  2.,  5.,  5.,  4.,  2.,  4.,  3.,  2.,  4.,  5.,
         5.,  5.,  2.,  6.,  3.,  6.,  2.,  1.,  3.,  2., -1.,  2.,  1.,
         2.,  3., -2., -3., -2., -2., -3.,  3.,  3.,  3.,  2.,  6.,  4.,
         5.,  5.,  5.,  5.,  5.,  5.,  3.,  3.,  4.

In [36]:
x_train, y_train = next(train_generator.generator())

In [8]:
np.asarray(X).shape

(2397, 400)

In [9]:
X[0]

array([ 1., -1.,  2.,  0.,  0.,  3.,  0.,  1., -3., -2.,  2.,  1.,  0.,
        0.,  3.,  5.,  5.,  5.,  5.,  5.,  4.,  3.,  5.,  5.,  5.,  5.,
        4.,  1.,  5.,  0.,  2.,  6.,  4.,  5.,  4.,  6.,  4.,  2.,  2.,
       -3., -1., -2.,  4.,  5.,  1., -1., -2., -4., -1.,  4.,  3.,  6.,
        4.,  6.,  3.,  4.,  0.,  5.,  5.,  3., -3., -1.,  0., -3.,  1.,
        0., -3., -2.,  4., -2.,  0.,  4.,  2.,  3., -2., -3., -2.,  1.,
        4.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.,  1.,  1.,
       -2., -3., -2., -2., -2.,  1.,  3.,  5.,  5.,  5.,  5.,  5.,  5.,
        3.,  4.,  1.,  1.,  0.,  2.,  5.,  5.,  5.,  5.,  4.,  1.,  5.,
        4.,  3.,  6.,  1.,  5.,  5.,  5.,  2.,  5.,  5.,  5.,  6.,  5.,
        5.,  6.,  4.,  2.,  5.,  5.,  4.,  2.,  4.,  3.,  2.,  4.,  5.,
        5.,  5.,  2.,  6.,  3.,  6.,  2.,  1.,  3.,  2., -1.,  2.,  1.,
        2.,  3., -2., -3., -2., -2., -3.,  3.,  3.,  3.,  2.,  6.,  4.,
        5.,  5.,  5.,  5.,  5.,  5.,  3.,  3.,  4.,  2.,  2.,  4

In [5]:
def model():
    model = Sequential()
    model.add(Conv1D(128, 5, activation='relu', padding='same', input_shape=(400, 1)))
    model.add(Conv1D(1, 1, padding='same'))
    model.summary()
    sgd = SGD(lr=0.01)
    model.compile(loss='mean_squared_error', optimizer=sgd, metrics=[metrics.mae, metrics.mse])
    return model

In [6]:
model_normal = model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_1 (Conv1D)            (None, 400, 128)          768       
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 400, 1)            129       
Total params: 897
Trainable params: 897
Non-trainable params: 0
_________________________________________________________________


In [7]:
X = np.asarray(X)
Y = np.asarray(Y)

KeyboardInterrupt: 